In [7]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>")) #makes the display bars longer and spread out 100% across the screen width

In [1]:
from vivarium import Artifact
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from db_queries import get_ids, get_outputs
import scipy.stats

In [2]:
#find file paths
%cd /share/costeffectiveness/results/vivarium_gates_bep/malnutrition/pakistan/2020_02_05_03_05_43
!ls -halt

/ihme/costeffectiveness/results/vivarium_gates_bep/malnutrition/pakistan/2020_02_05_03_05_43
total 12K
drwxrwxr-x 3 collijk Domain Users 512 Feb  5 18:00 .
drwxr-xr-x 2 collijk Domain Users 512 Feb  5 03:19 __pycache__
drwxrwxr-x 3 collijk Domain Users 512 Feb  5 03:07 ..


In [3]:
output_dirs = ['/share/costeffectiveness/results/vivarium_gates_bep/malnutrition/india/2020_02_05_03_02_13/count_space/',
              '/share/costeffectiveness/results/vivarium_gates_bep/malnutrition/mali/2020_02_05_03_02_02/count_space/',
              '/share/costeffectiveness/results/vivarium_gates_bep/malnutrition/tanzania/2020_02_05_03_07_37/count_space/']

locations = ['India','Mali','Tanzania']

In [4]:
master_stunting = pd.DataFrame()
master_wasting = pd.DataFrame()

for i in list(range(len(output_dirs))):
    cgf = pd.read_hdf(output_dirs[i] + 'category_counts.hdf').reset_index()
    cgf = (cgf.where(cgf['sex'] == 'male').dropna()
           .groupby(['input_draw','cgf_risk','risk_category','malnourishment_category']).sum()
           .reset_index())
    pop = (cgf.where(cgf['cgf_risk'] == 'stunting').dropna()
          .groupby(['input_draw','malnourishment_category'], as_index=False).sum()
          .rename(columns={'category_counts':'population'}))
    cgf = cgf.where(cgf['risk_category'] == ('cat1' or 'cat2'))
    data = cgf.merge(pop, right_on=['input_draw','malnourishment_category'], 
                     left_on=['input_draw','malnourishment_category'])
    stratified = data.groupby(['input_draw','malnourishment_category','cgf_risk']).sum().reset_index()
    overall = data.groupby(['input_draw','cgf_risk']).sum().reset_index()
    overall['malnourishment_category'] = 'all'
    data = stratified.append(overall)  
    
    stunted = (data.where(data['cgf_risk'] == 'stunting').dropna())
    stunted['percent'] = stunted['category_counts'] / stunted['population']
    stunted = (stunted.groupby(['malnourishment_category']).mean()
              .drop(columns=['input_draw','category_counts','population'])
               .reset_index()) 
    stunted['location'] = locations[i]
    master_stunting = master_stunting.append(stunted)   
    
    wasted = (data.where(data['cgf_risk'] == 'wasting').dropna())
    wasted['percent'] = wasted['category_counts'] / wasted['population']
    wasted = (wasted.groupby(['malnourishment_category']).mean()
              .drop(columns=['input_draw','category_counts','population'])
               .reset_index()) 
    wasted['location'] = locations[i]
    master_wasting = master_wasting.append(wasted)
    master_wasting

/share/code/nicoly/miniconda/envs/gates_bep/lib/python3.6/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [5]:
master_wasting

,malnourishment_category,percent,location
0,all,0.055677,India
1,malnourished,0.142786,India
2,not_malnourished,0.038791,India
0,all,0.030601,Mali
1,malnourished,0.090116,Mali
2,not_malnourished,0.024041,Mali
0,all,0.014787,Tanzania
1,malnourished,0.041370,Tanzania
2,not_malnourished,0.012091,Tanzania


In [6]:
master_stunting

,malnourishment_category,percent,location
0,all,0.091158,India
1,malnourished,0.200051,India
2,not_malnourished,0.069993,India
0,all,0.038440,Mali
1,malnourished,0.131865,Mali
2,not_malnourished,0.028073,Mali
0,all,0.059996,Tanzania
1,malnourished,0.138289,Tanzania
2,not_malnourished,0.052009,Tanzania
